<a href="https://colab.research.google.com/github/imsaurab/Oanda-with-python/blob/main/Automated_bot_Oanda_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
API = "api-fxpractice.oanda.com"
STREAM_API = "stream-fxpractice.oanda.com"

In [ ]:
ACCESS_TOKEN ="d3bc3d87ed3dae4c722e6205ac5aeea6-fe7755fc3e146ad278186b9130ee5e4d"


ACCOUNT_ID =None

# Accessing API

In [ ]:
import requests
import json
import numpy as np
import pandas as pd

In [ ]:
PRICING_PATH = f"/v3/accounts/101-001-17640366-001/pricing"

In [ ]:
query = {"instruments": "USD_CAD"} 
headers = {"Authorization": "Bearer "+ "d3bc3d87ed3dae4c722e6205ac5aeea6-fe7755fc3e146ad278186b9130ee5e4d"
}

In [ ]:
response = requests.get("https://"+ "api-fxpractice.oanda.com"+PRICING_PATH, headers=headers, params=query)

In [ ]:
response.json()

{'errorMessage': "Invalid value specified for 'from'. Time is in the future"}

In [ ]:
def get_ask_price(instrument):
    """
    Returns a (time, ask price) tuple.
    """
    query = {"instruments": instrument} 
    response = requests.get("https://"+"api-fxpractice.oanda.com"+PRICING_PATH, headers=headers, params=query)
    json_response = response.json()

    prices = json_response["prices"][0]["asks"][0]["price"]
    prices = float(prices)

    time_str = json_response["time"]
    time = pd.to_datetime(time_str)

    return time, prices

In [ ]:
get_ask_price("USD_CAD")

(Timestamp('2021-01-13 16:31:54.181849034+0000', tz='UTC'), 1.27224)

In [ ]:
get_ask_price("EUR_USD")

(Timestamp('2021-01-13 16:32:00.261717738+0000', tz='UTC'), 1.21587)

# 

In [ ]:
pip install v20

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 184kB 7.1MB/s 
  Created wheel for v20: filename=v20-3.0.25.0-cp36-none-any.whl size=85761 sha256=2dd3cd20fee18544d2d2f17528e89197b3f4693f76179468b6cd82ac5b616ec3
  Stored in directory: /root/.cache/pip/wheels/2e/e6/cf/5d1e2fc46636f8ac745eda7bc5a859ae0c5072e07ce0023e0f
Successfully built v20


# Using  v20

https://github.com/oanda/v20-python

In [ ]:
import v20

In [ ]:
api = v20.Context(hostname= "api-fxpractice.oanda.com", token="d3bc3d87ed3dae4c722e6205ac5aeea6-fe7755fc3e146ad278186b9130ee5e4d", port=443)
stream_api = v20.Context(hostname="stream-fxpractice.oanda.com", token="d3bc3d87ed3dae4c722e6205ac5aeea6-fe7755fc3e146ad278186b9130ee5e4d", port=443)

In [ ]:
response = api.pricing.get(accountID="101-001-17640366-001", instruments="USD_CAD,EUR_CAD")

In [ ]:
json.loads(response.body["prices"][0].json())

{'asks': [{'liquidity': 10000000, 'price': 1.27214}],
 'bids': [{'liquidity': 10000000, 'price': 1.27195}],
 'closeoutAsk': 1.27214,
 'closeoutBid': 1.27195,
 'instrument': 'USD_CAD',
 'quoteHomeConversionFactors': {'negativeUnits': '0.78619443',
  'positiveUnits': '0.786077'},
 'status': 'tradeable',
 'time': '2021-01-13T15:51:05.595249262Z',
 'tradeable': True,
 'type': 'PRICE',
 'unitsAvailable': {'default': {'long': '4999998.0', 'short': '4999998.0'},
  'openOnly': {'long': '4999998.0', 'short': '4999998.0'},
  'reduceFirst': {'long': '4999998.0', 'short': '4999998.0'},
  'reduceOnly': {'long': '0.0', 'short': '0.0'}}}

# Streaming

In [ ]:
import time 

In [ ]:
from_time = time.mktime(pd.to_datetime("12/01/2021").timetuple())
to_time = time.mktime(pd.to_datetime("14/01/2021").timetuple())
print(f"Window start: {from_time}, window end: {to_time}." )

Window start: 1638316800.0, window end: 1610582400.0.


In [ ]:
header = {"Authorization": "Bearer "+"d3bc3d87ed3dae4c722e6205ac5aeea6-fe7755fc3e146ad278186b9130ee5e4d"} 
query = {"from": str(from_time), "to": str(to_time), "granularity": "D"}

In [ ]:
INSTRUMENT = "EUR_USD"

In [ ]:
CANDLES_PATH = f"/v3/accounts/101-001-17640366-001/instruments/EUR_USD/candles"

In [ ]:
response = requests.get("https://"+ "api-fxpractice.oanda.com"+CANDLES_PATH, headers=header, params=query)

In [ ]:
def json_to_pandas(json):
    json_file = json.json()
    price_json = json_file["candle"]
    times = []
    close_price, high_price, low_price, open_price = [], [], [], []

    for candles in price_json:
        times.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
    
    dataframe = pd.DataFrame({"close": close_price, "high": high_price, "low": low_price, "open": open_price})
    dataframe.index = pd.to_datetime(times)
    return dataframe

In [ ]:
EURCAD_df = json_to_pandas(response)

In [ ]:
EURCAD_df.head(5)

In [ ]:
EURCAD_df["close"].plot()

In [ ]:
stream_api

'stream-fxpractice.oanda.com'

In [ ]:
response = stream_api.pricing.stream(accountID="101-001-17640366-001", instruments="EUR_USD", snapshot=True)

In [ ]:
for price_data in response.lines:
    print(price_data)

b'{"type":"PRICE","time":"2021-01-14T11:56:50.552091471Z","bids":[{"price":"1.21505","liquidity":10000000}],"asks":[{"price":"1.21519","liquidity":10000000}],"closeoutBid":"1.21505","closeoutAsk":"1.21519","status":"tradeable","tradeable":true,"instrument":"EUR_USD"}'
b'{"type":"PRICE","time":"2021-01-14T11:56:54.815362789Z","bids":[{"price":"1.21506","liquidity":10000000}],"asks":[{"price":"1.21520","liquidity":10000000}],"closeoutBid":"1.21506","closeoutAsk":"1.21520","status":"tradeable","tradeable":true,"instrument":"EUR_USD"}'
b'{"type":"PRICE","time":"2021-01-14T11:56:56.086192242Z","bids":[{"price":"1.21504","liquidity":10000000}],"asks":[{"price":"1.21519","liquidity":10000000}],"closeoutBid":"1.21504","closeoutAsk":"1.21519","status":"tradeable","tradeable":true,"instrument":"EUR_USD"}'
b'{"type":"HEARTBEAT","time":"2021-01-14T11:56:58.655135286Z"}'
b'{"type":"PRICE","time":"2021-01-14T11:56:59.807483301Z","bids":[{"price":"1.21506","liquidity":10000000}],"asks":[{"price":"1.2

KeyboardInterrupt: ignored